<a href="https://colab.research.google.com/github/rameshbabulakshmanan84/NLP-series/blob/master/Aug_3_Clinicalbert-5000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [3]:
import pandas as pd
import csv
path='/content/drive/My Drive/NLP/ClinicalBiobert/Clinicalbert-5000_Aug3/BC2GM-IOB-Train_5000.xlsx'
data=pd.read_excel(path,delimiter='\t',names=["Words","tags"],encoding='unicode_escape')

In [4]:
data.shape

(5011, 2)

In [5]:
data['tags'].isnull().sum()

1

In [6]:
data['tags']=data['tags'].fillna('O')

In [7]:
data['tags'].isnull().sum()
data['tags'].unique()

array(['O', 'B-GENE', 'I-GENE', 'B-DISEASE', 'I-DISEASE', 'B-DRUGS',
       'I-DRUGS'], dtype=object)

Concatenate the words into a sentence - Make sure we convert by using str(input)

In [8]:
sentences1=[]
tags1=[]
words=data['Words'].values.tolist()
tag=data['tags'].values.tolist()
#words
temp_sen=[]
temp_tag=[]
count=0
for i in words:
  #print(i)
  
  if i != 'End':
    temp_sen.append(str(i))
    temp_tag.append(str(tag[count]))
    count+=1
  else:
    sentences1.append(temp_sen)
    tags1.append(temp_tag)
    temp_sen=[]
    temp_tag=[]

In [9]:
 #check the data type pf the sentences
 type(sentences1[0][2])

str

In [10]:
#Print the length of tags1 and sentences1
print(len(tags1))
print(len(sentences1))

168
168


In [11]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [12]:
#map the tags with integers -> tags refer to ner
#data["tag"]:retrieves unique tags, set(data["tag"]) -> refers to set of distinct values
tag_values=list(set(data["tags"].values))
tag_values.append("PAD")
#create a dictionary of tag and values
tag2idx={t:i for i,t in enumerate(tag_values)}

In [13]:
print(tag2idx)
#Tags available :
# I-GENE,B-GENE

{'I-DRUGS': 0, 'B-GENE': 1, 'I-GENE': 2, 'I-DISEASE': 3, 'B-DRUGS': 4, 'O': 5, 'B-DISEASE': 6, 'PAD': 7}


Set up the transformers library

In [14]:
!pip3 install transformers


     |████████████████████████████████| 778kB 4.7MB/s 
     |████████████████████████████████| 3.0MB 13.8MB/s 
     |████████████████████████████████| 890kB 43.3MB/s 
     |████████████████████████████████| 1.1MB 51.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=f6cd56ef352f2a6a730e7cb9e236c9f5b865ba6dd7c3151a2a4faf2c39595328
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [15]:
#use pytorch implementation of bert
import torch
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
from transformers import AutoTokenizer,AutoModelForTokenClassification,AutoModel
import logging
logging.basicConfig(level=logging.INFO)

In [17]:
#import clinicalbiobert
clibert_tokenizer=AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
clibert_model=AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/emilyalsentzer/Bio_ClinicalBERT/config.json from cache at /root/.cache/torch/transformers/f1b766d4011bf16cfd52c7be6150a4f5d4c2256eaca3c2edb073f468005f08a2.e99f38979b92a72e708709cb38397fa864caf99dd2e2ed75826560450aa14a0c
INFO:transformers.configuration_utils:Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

INFO:transformers.tokenization_utils_base:Model name 'emilyalsentzer/Bio_ClinicalBERT' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased

In [18]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [19]:
#Set the max length and batchsize
MAX_LEN = 75
Batch_Size = 32

In [65]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu=torch.cuda.device_count()

In [22]:
def token_and_preserve_labels(sentence,text_labels):
  tokenized_sentence=[]
  labels=[]

  for word,label in zip(sentence,text_labels):
    #tokenize the word and count of # of subwords that word is toeknized into 
    #demonstrators ['demons', '##tra', '##tors']
    tokenized_word=scibert_tokenizer.tokenize(word)
    n_subwords=len(tokenized_word)

    # Add the tokenized word to the final tokenized word list
    tokenized_sentence.extend(tokenized_word)

    # Add the same label to the new list of labels `n_subwords` times
    labels.extend([label] * n_subwords)

  return tokenized_sentence,labels 

In [23]:
#sentence and corresponsing labels 
#sentence form : [list of words]
#labels form : #labels form [ list of ner tags]
tokenized_texts_and_labels=[token_and_preserve_labels(sent,labels) for sent,labels in zip(sentences1,tags1)]

In [24]:
print(tokenized_texts_and_labels)

[(['im', '##mu', '##no', '##his', '##to', '##chemical', 'stain', '##ing', 'was', 'positive', 'for', 's', '-', '100', 'in', 'all', '9', 'cases', 'stained', ',', 'positive', 'for', 'h', '##mb', '-', '45', 'in', '9', '(', '90', '%', ')', 'of', '10', ',', 'and', 'negative', 'for', 'c', '##yt', '##oker', '##ati', '##n', 'in', 'all', '9', 'cases', 'in', 'which', 'my', '##x', '##oid', 'me', '##lan', '##oma', 'remained', 'in', 'the', 'block', 'after', 'previous', 'sections', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'B-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DISEASE', 'B-DISEASE', 'B-DISEASE', 'I-DISEASE', 'I-DISEASE', 'I-DISEASE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['ch', '##lora', '##mp', '##hen', '##ico', '##l', 'ace', '##ty', '##lt', '##ran', '##s', '##fer

In [25]:
# split text and labels
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]

labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [26]:
print(tokenized_texts)

[['im', '##mu', '##no', '##his', '##to', '##chemical', 'stain', '##ing', 'was', 'positive', 'for', 's', '-', '100', 'in', 'all', '9', 'cases', 'stained', ',', 'positive', 'for', 'h', '##mb', '-', '45', 'in', '9', '(', '90', '%', ')', 'of', '10', ',', 'and', 'negative', 'for', 'c', '##yt', '##oker', '##ati', '##n', 'in', 'all', '9', 'cases', 'in', 'which', 'my', '##x', '##oid', 'me', '##lan', '##oma', 'remained', 'in', 'the', 'block', 'after', 'previous', 'sections', '.'], ['ch', '##lora', '##mp', '##hen', '##ico', '##l', 'ace', '##ty', '##lt', '##ran', '##s', '##fer', '##ase', 'ass', '##ays', 'examining', 'the', 'ability', 'of', 'i', '##e', '##86', 'to', 're', '##press', 'activity', 'from', 'the', 'h', '##c', '##m', '##v', 'major', 'i', '##e', 'promoter', 'or', 'activate', 'the', 'h', '##c', '##m', '##v', 'early', 'promoter', 'for', 'the', '2', '.', '2', '-', 'k', '##b', 'class', 'of', 'r', '##nas', 'demonstrated', 'the', 'functional', 'integrity', 'of', 'the', 'i', '##e', '##86', 'pro

In [27]:
print(labels)

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'B-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DISEASE', 'B-DISEASE', 'B-DISEASE', 'I-DISEASE', 'I-DISEASE', 'I-DISEASE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'O'], ['O', 'O', 'O', 'O', 'O', 'B-GENE', 'I-GENE', 'I-GENE', 'I-GENE', 'I-GENE', 'I-GENE', 'I-GENE', 'I-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', '

In [28]:
input_ids = pad_sequences([scibert_tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [29]:
print(len(input_ids))

168


In [30]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [31]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [32]:
print(attention_masks)

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [33]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids,tags,random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=2018, test_size=0.1)

In [34]:
#convert all tensor format
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [35]:
tr_tags.shape

torch.Size([151, 75])

In [37]:
val_tags.shape

torch.Size([17, 75])

In [38]:
tr_inputs.shape

torch.Size([151, 75])

In [39]:
#train_data in the form - input tokens, attention masks for the corresponding tokens and the tags for the tokens
train_data = TensorDataset(tr_inputs,tr_masks,tr_tags)
#returns random indices from the data source
train_sampler = RandomSampler(train_data)
#dataloader combines the dataset and the sampler 
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)
valid_data = TensorDataset(val_inputs, val_masks, val_tags)
#samples elements sequentially in the same order
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data,sampler=valid_sampler, batch_size=32)

In [40]:
for input,mask,tag in train_dataloader:
  print(mask.shape)
  print(input.shape)
  print(tag.shape)

torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([32, 75])
torch.Size([23, 75])
torch.Size([23, 75])
torch.Size([23, 75])


Setup the Bert model for finetuning

In [41]:
import transformers
#bertfortokenclassification addsa classifier on top of bert model
from transformers import BertForTokenClassification,AdamW,AutoConfig

In [42]:
model1 = BertForTokenClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",num_labels=len(tag2idx), output_attentions = False,output_hidden_states = False)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/emilyalsentzer/Bio_ClinicalBERT/config.json from cache at /root/.cache/torch/transformers/f1b766d4011bf16cfd52c7be6150a4f5d4c2256eaca3c2edb073f468005f08a2.e99f38979b92a72e708709cb38397fa864caf99dd2e2ed75826560450aa14a0c
INFO:transformers.configuration_utils:Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7
  },
  "layer_norm_eps": 1e-12,
  "max_position_embe

In [43]:
import torch
#specify GPU as device
if torch.cuda.is_available():
  device=torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:125: UserWarning: 
Tesla T4 with CUDA capability sm_75 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the Tesla T4 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [44]:
model1.num_labels

8

In [46]:
# Tell pytorch to run this model on the GPU.
model1.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [48]:
#Print the different parameters
params=list(model1.named_parameters())

#print
print("the model has {:} number of parameters".format(len(params)))

the model has 201 number of parameters


In [49]:
for p in params[0:5]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print("---First Transformer---")
for p in params[5:21]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
for p in params[190:200]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print("---output layer---")
for p in params[-4:]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

bert.embeddings.word_embeddings.weight                  (28996, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)
---First Transformer---
bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (768,)
bert.encoder.layer.0.attention.output.LayerNorm.weight        (768,)
bert.encod

In [50]:
#grab the optimzer
optimizer=AdamW(model1.parameters(),
               lr=2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5 
               eps=1e-8) #a very small number to prevent any division by zero in the implementation)

In [51]:
from transformers import get_linear_schedule_with_warmup

In [53]:
epochs=4

In [54]:
total_steps = len(train_dataloader) * epochs

In [55]:
#Create a schedule with a learning rate that decreases linearly from the initial lr set in the optimizer to 0, after a warmup period during which it increases linearly from 0 to the initial lr set in the optimizer.

scheduler=get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,#deafault value
    num_training_steps=total_steps)


In [56]:
from sklearn.metrics import f1_score, accuracy_score

In [57]:
import random
import numpy as np

In [58]:
# Set the seed value all over the place to make this reproducible.
seed_val=42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [60]:
#for ech epoch
#Run the training for the given epochs - typically between 2 and 4
#range 0 to 3

loss_values, validation_loss_values = [], []


for _ in trange(epochs,desc="EPOCH"):
  print("")
  #print ---Epoch 1/2 ----
  #print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
  print('Training...')

  #Reset the total loss for this epoch
  total_train_loss=0
  
  #train the model
  model1.train()

  for step,batch in enumerate(train_dataloader):
    print(step)
    #print(batch)

    #add the batch to the GPU
    batch=tuple(t.to(device) for t in batch)
    #print(batch.shape)
    b_input_ids,b_input_mask,b_labels=batch
    #print(b_labels)
  #always clear the gradients before doing a backward pass
    model1.zero_grad()

   #perform a forward pass
    loss,logits=model1(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)
    
    #logits = logits.detach().cpu().numpy()
    #print("step number -", step)
    #print("score--",logits.shape)
    #outout shape in the form : 32*75*18 --32 batch size,75 --> sequence size, 18 --> Tags 
    #print("argmax--")
    #print("logits--",logits)
    #print("argmax--",np.argmax(logits,axis=1))
   #calculate the accumulating loss
   #`loss` is a Tensor containing a # single value; the `.item()` function just returns the Python value 
    total_train_loss+=loss.item()

   # Perform a backward pass to calculate the gradients.
   #compute gradient of loss w.r.t all the parameters in loss that have requires_grad = True and store them in parameter.grad attribute for every parameter.
    loss.backward()

   # Clip the norm of the gradients to 1.0.
   # This is to help prevent the "exploding gradients" problem
   #within a specific range (clip)
    torch.nn.utils.clip_grad_norm_(model1.parameters(), 1.0)

     
   # Update parameters and take a step using the computed gradient.
   # The optimizer dictates the "update rule"--how the parameters are
   # modified based on their gradients, the learning rate, etc.
   #updates all the parameters based on parameter.grad
    optimizer.step()

   # Update the learning rate.
    scheduler.step()
#Return to the epoch
   # Calculate the average loss over all of the batches.  
  avg_train_loss= total_train_loss/len(train_dataloader)
  print("Average train loss: {}".format(avg_train_loss))
  #append the loss for trainig
  loss_values.append(avg_train_loss)
  print(" Training is completed")
  print("Running Validation... ")

  #put the model in evaluation mode
  model1.eval()

  # Tracking variables 
  total_eval_accuracy = 0
  total_eval_loss = 0
  nb_eval_steps = 0
  predictions , true_labels = [], []

  for batch in valid_dataloader:

    #add the batch to the GPU
    batch=tuple(t.to(device) for t in batch)
    b_input_ids,b_input_mask,b_labels=batch
   
    #print("validation labels:",b_labels)
     #always clear the gradients
     #in evaluation mode, you can't do backprog, to save time, dont compute the gradients
    with torch.no_grad():
      #perform a forward pass
      loss,logits=model1(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)

    logits=logits.detach().cpu().numpy()
    label_ids=b_labels.to('cpu').numpy()
    label_ids=b_labels


   #calculate the accumulating loss
   #`loss` is a Tensor containing a # single value; the `.item()` function just returns the Python value 
    total_eval_loss+=loss.mean().item()
    predictions.extend([list(p) for p in np.argmax(logits,axis=2)])
    true_labels.extend(label_ids)

  eval_loss=total_eval_loss/len(valid_dataloader) 
  validation_loss_values.append(eval_loss)
  print("Validation loss: {}".format(eval_loss))

  pred_tags=[tag_values[p_i] for p,l in zip(predictions,true_labels) for p_i,l_i in zip(p,l) if tag_values[l_i] != "PAD"]

  valid_tags=[tag_values[l_i] for l in true_labels for l_i in l if tag_values[l_i]!="PAD"]

  print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
  print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags,average='weighted')))

EPOCH:   0%|          | 0/4 [00:00<?, ?it/s]


Training...
0
1
2
3
4


EPOCH:  25%|██▌       | 1/4 [00:02<00:08,  2.86s/it]

Average train loss: 1.1998387217521667
 Training is completed
Running Validation... 
Validation loss: 0.7127617001533508
Validation Accuracy: 0.8625180897250362
Validation F1-Score: 0.9261849261849263

Training...
0
1
2
3
4


EPOCH:  50%|█████     | 2/4 [00:05<00:05,  2.73s/it]

Average train loss: 0.6933956861495971
 Training is completed
Running Validation... 
Validation loss: 0.6523022651672363
Validation Accuracy: 0.8625180897250362
Validation F1-Score: 0.9261849261849263

Training...
0
1
2
3
4


EPOCH:  75%|███████▌  | 3/4 [00:07<00:02,  2.64s/it]

Average train loss: 0.6620024919509888
 Training is completed
Running Validation... 
Validation loss: 0.6455438137054443
Validation Accuracy: 0.8625180897250362
Validation F1-Score: 0.9261849261849263

Training...
0
1
2
3
4


EPOCH: 100%|██████████| 4/4 [00:10<00:00,  2.54s/it]

Average train loss: 0.6409375429153442
 Training is completed
Running Validation... 
Validation loss: 0.6469035744667053
Validation Accuracy: 0.8625180897250362
Validation F1-Score: 0.9261849261849263


Demostrate the testing of sentence 

In [66]:
test_sentence = "To study the safety and efficacy of  methylphenidate in children with the dual diagnosis of  epilepsy and attention deficit hyperactivity disorder  and double-blind, crossover design was used to compare the effects "

In [67]:
#tokenize  the text
tokenized_sentence=scibert_tokenizer.encode(test_sentence)
#the output is sequence of ids (token to ids)

In [70]:
print(tokenized_sentence)

[101, 1106, 2025, 1103, 3429, 1105, 23891, 1104, 1899, 18873, 27801, 2605, 9216, 1107, 1482, 1114, 1103, 7058, 12645, 1104, 174, 24898, 12685, 1105, 2209, 16312, 177, 24312, 11179, 6366, 8936, 1105, 2702, 118, 7198, 117, 18847, 1902, 1108, 1215, 1106, 14133, 1103, 3154, 102]


In [68]:
#conver to tensor format
input_ids = torch.tensor([tokenized_sentence])

In [76]:
# Tell pytorch to run this model on the GPU.
model1.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [77]:
print(input_ids)

tensor([[  101,  1106,  2025,  1103,  3429,  1105, 23891,  1104,  1899, 18873,
         27801,  2605,  9216,  1107,  1482,  1114,  1103,  7058, 12645,  1104,
           174, 24898, 12685,  1105,  2209, 16312,   177, 24312, 11179,  6366,
          8936,  1105,  2702,   118,  7198,   117, 18847,  1902,  1108,  1215,
          1106, 14133,  1103,  3154,   102]])


In [83]:
use_cuda = torch.cuda.is_available()
print(use_cuda)

True


In [80]:
if use_cuda:
  input_ids.cuda()


In [85]:
#this 
input_ids=input_ids.cuda()

In [ ]:
 #batch=tuple(t.to(device) for t in batch)

In [94]:
#run the sentence through the model
with torch.no_grad():
  logits=model1(input_ids)

In [96]:
label_indices=torch.argmax(logits[0],axis=2)


In [97]:
print(label_indices)

tensor([[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
         5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]],
       device='cuda:0')


In [98]:
#join the split tokens
tokens=scibert_tokenizer.convert_ids_to_tokens(input_ids[0])

In [99]:
print(tokens)

['[CLS]', 'to', 'study', 'the', 'safety', 'and', 'efficacy', 'of', 'met', '##hyl', '##phe', '##ni', '##date', 'in', 'children', 'with', 'the', 'dual', 'diagnosis', 'of', 'e', '##pile', '##psy', 'and', 'attention', 'deficit', 'h', '##yper', '##act', '##ivity', 'disorder', 'and', 'double', '-', 'blind', ',', 'crossover', 'design', 'was', 'used', 'to', 'compare', 'the', 'effects', '[SEP]']


In [100]:
new_token,new_labels=[],[]
for token,label_idx in zip(tokens,label_indices[0]):
  if token.startswith("##"):
    #add at the end
    new_token[-1]=new_token[-1]+token[2:]
  else:
    #if its not special token
    new_labels.append(tag_values[label_idx])
    new_token.append(token) 

In [101]:
for token,label in zip(new_token,new_labels):
  print("{}\t{}".format(token,label))
  

[CLS]	O
to	O
study	O
the	O
safety	O
and	O
efficacy	O
of	O
methylphenidate	O
in	O
children	O
with	O
the	O
dual	O
diagnosis	O
of	O
epilepsy	O
and	O
attention	O
deficit	O
hyperactivity	O
disorder	O
and	O
double	O
-	O
blind	O
,	O
crossover	O
design	O
was	O
used	O
to	O
compare	O
the	O
effects	O
[SEP]	O
